In [1]:
USE_MOCK_PROPERTY_MODEL = True

In [2]:
import sys
from pprint import pprint

# project dependencies
proj_path = 'd:/code/Projects/property_mapping_sql_generator/'
sys.path.insert(0, proj_path)
from dsl.engine import *

In [3]:
# simple case where every non inline mapping is for the same ref col
# and search values are uniform between case statements
p = Property(Case('ref', [('x', 'p1'), ('y', 'p2')], default='pN'))
a = Analysis(Case('ref', [('x', 'a1'), ('y', 'a2')], default='aN'))
i = NoIndicator()
u = Uom(Case('ref', [('x', 'u1'), ('y', 'u2')], default='uN'))
r = NoRatio()

print(f"Property logic branches:")
pprint(list(p.get_branches()))

print(f"\nAnalysis logic branches:")
pprint(list(a.get_branches()))

print(f"\nUom logic branches:")
pprint(list(u.get_branches()))

Property logic branches:
[LogicBranch(result='p1', search_expr='x', ref='a.ref'),
 LogicBranch(result='p2', search_expr='y', ref='a.ref'),
 LogicBranch(result='pN', search_expr=None, ref='a.ref')]

Analysis logic branches:
[LogicBranch(result='a1', search_expr='x', ref='a.ref'),
 LogicBranch(result='a2', search_expr='y', ref='a.ref'),
 LogicBranch(result='aN', search_expr=None, ref='a.ref')]

Uom logic branches:
[LogicBranch(result='u1', search_expr='x', ref='a.ref'),
 LogicBranch(result='u2', search_expr='y', ref='a.ref'),
 LogicBranch(result='uN', search_expr=None, ref='a.ref')]


In [4]:
expander = MappingExpander(p, a, i, u, r)

# need to register the property model (or a mock for this to work)
if USE_MOCK_PROPERTY_MODEL:
    from tests.dependency_setup import register_test_dependencies
    register_test_dependencies()
else:
    dependencies.register_dependencies()
    
# need to initialise this after the property model is loaded in above
row = MappingRow(name="No name", prop=p, anal=a, uom=u, ind=i, ratio=r)

In [5]:
# measurement descriptor (analysis, property, indicator)
mds_expanded = list(expander.expand_mappings(use_default_value_descriptors=True))

print("The analysis, property and indicator have been expanded to the headings:")
for header_elements in sorted(mds_expanded):
    print(row.format_header(header_elements))

# uom/ratio will show as unable to id as these are not real properties 
# so do not exist in the property model - not exanding on uom/ratio 
# requires us to be able to get the defaults from the prop model

The analysis, property and indicator have been expanded to the headings:
p1.a1<unable to identify correct uom>
p2.a2<unable to identify correct uom>
pN.aN<unable to identify correct uom>


In [6]:
all_expanded = list(expander.expand_mappings(use_default_value_descriptors=False))

print("Expect 3 results because there is 1 ref col with 3 branches with unique search terms (incl default)")

print("\nThe anal, prop, ind, uom & ratio have been expanded to the headings:")
print("-" * 70)
for header_elements in sorted(all_expanded):
    print(row.format_header(header_elements))

Expect 3 results because there is 1 ref col with 3 branches with unique search terms (incl default)

The anal, prop, ind, uom & ratio have been expanded to the headings:
----------------------------------------------------------------------
p1.a1<u1>
p2.a2<u2>
pN.aN<uN>


In [7]:
display(sorted(all_expanded))

[HeaderElements(prop='p1', anal='a1', uom='u1', ind='na', ratio='NoRatio'),
 HeaderElements(prop='p2', anal='a2', uom='u2', ind='na', ratio='NoRatio'),
 HeaderElements(prop='pN', anal='aN', uom='uN', ind='na', ratio='NoRatio')]